In [1]:
import pandas as pd
import sys; sys.path.insert(1, '../../../../')
from utils import utility 

pricesDataFolder = "../../../../prices_data/"
dfH4 = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"H4/[SP500]_H4_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)

dfH4 = dfH4.rename(
    columns = {
        '<CLOSE>':'close',
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low'
    }
)

dfH4['datetime'] = pd.to_datetime(dfH4['<DATE>'] + ' ' + dfH4['<TIME>'])
dfH4 = dfH4.drop(['<TICKVOL>', '<VOL>', '<SPREAD>', '<DATE>', '<TIME>'], axis=1)
dfH4 = utility.heikinashi(dfH4) #add heikin ashi candles
dfH4["shortTermMA"] = dfH4["close"].rolling(window=50).mean() # add moving average 50
dfH4 = dfH4.drop(columns=["datetime"])
dfH4 = dfH4[["close", "open","high", "low", "shortTermMA", "HA close","HA open", "HA high", "HA low"]]
data = dfH4.iloc[50:].to_numpy()

In [2]:
print(dfH4)
print(data.shape)

         close     open     high      low  shortTermMA   HA close   HA open  \
0      1856.90  1858.90  1861.50  1854.90          NaN  1858.0500  1857.900   
1      1856.10  1856.70  1858.40  1853.20          NaN  1856.1000  1857.900   
2      1857.80  1856.10  1860.80  1854.00          NaN  1857.1750  1856.400   
3      1857.80  1857.80  1860.30  1852.70          NaN  1857.1500  1856.950   
4      1872.60  1857.60  1873.60  1855.60          NaN  1864.8500  1857.800   
...        ...      ...      ...      ...          ...        ...       ...   
15421  5138.60  5147.69  5154.58  5137.61    5144.6264  5144.6200  5144.425   
15422  5136.94  5138.84  5143.79  5122.34    5145.4218  5135.4775  5143.145   
15423  5170.60  5136.69  5175.13  5132.38    5146.6928  5153.7000  5137.890   
15424  5176.05  5170.85  5180.54  5163.05    5147.6494  5172.6225  5153.645   
15425  5170.13  5175.16  5176.18  5168.64    5148.0044  5172.5275  5173.450   

       HA high   HA low  
0      1861.50  1854.90  

In [3]:
import tensorflow.compat.v1 as tfv1

config = tfv1.ConfigProto()
config.gpu_options.allow_growth = True
session = tfv1.Session(config=config)

2024-04-28 18:40:29.359576: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 18:40:30.482115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-28 18:40:31.745366: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 18:40:31.932144: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 18:40:31.932238: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] co

In [2]:
import sys; sys.path.insert(1, '../../../../')

from reinforcement_learning.trading_agent.ppo.using_array.ac_training_loop_without_strat import actorCritictrainingLoop
from reinforcement_learning.trading_agent.ppo.agent_no_strat import Agent

params = {'sequence_length':100, 'h':4, 'd_k':1, 'd_v':1, 'd_model':4, 'd_ff':128, 'n':5}

agent = Agent(base_dir='../tmp/', chkpt_dir='model_last_trained', transParams=params, alpha=0.0001)
agent.load_weights()
score_history = actorCritictrainingLoop(data, agent, n_games=1000, save_model=True)

2024-04-28 18:52:40.497879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 18:52:41.195134: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-28 18:52:42.286746: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 18:52:42.317648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 18:52:42.317698: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] co

--- loading models weights ---


/home/tessan/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 174 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


--- saving model weights ---
episode:0, capital:4035.17, actions:[0, 2, 1], rewards:[45.1, 0.0, 35.2], avg_score:80.280, nb iter:3
episode:1, capital:4032.47, actions:[0, 2, 1], rewards:[0.8, 0.0, -2.7], avg_score:39.190, nb iter:3
episode:2, capital:4042.46, actions:[0, 1], rewards:[17.8, 10.0], avg_score:35.393, nb iter:2
episode:3, capital:4047.49, actions:[0, 1], rewards:[-2.2, 5.0], avg_score:27.245, nb iter:2
episode:4, capital:4079.29, actions:[0, 2, 2, 1], rewards:[21.6, 0.0, 0.0, 31.8], avg_score:32.476, nb iter:4
episode:5, capital:4011.42, actions:[0, 2, 2, 1], rewards:[-46.5, 0.0, 0.0, -67.9], avg_score:7.995, nb iter:4


KeyboardInterrupt: 

In [ ]:
import plotly.express as px
import tensorflow as tf

score_history = [tf.get_static_value(x) for x in score_history]

df = pd.DataFrame(dict(
    y = score_history,
    x = range(0, len(score_history))
))
fig = px.line(df, x="x", y="y", title="Agent score") 
fig.show()